In [784]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import operator
import time
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.utils.multiclass import type_of_target
import tensorflow as tf
from numpy import unique
from numpy import reshape
from tensorflow.keras.models import Sequential
from sklearn.model_selection import cross_val_score
from tensorflow.keras.layers import Input, Conv1D, Dense, Dropout, BatchNormalization, Flatten, MaxPooling1D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from scikeras.wrappers import KerasClassifier  # Use scikeras for scikit-learn compatibility
from math import floor
from bayes_opt import BayesianOptimization
from tensorflow.keras.layers import LeakyReLU  # Use tensorflow.keras instead of keras
LeakyReLU = LeakyReLU(negative_slope=0.1)
import warnings

In [785]:
path = r'D:\WORK\Bootcamp\Machine Learning\Achievement 1\Project\Data'

In [786]:
# Read the CSV file
df = pd.read_csv(os.path.join(path, 'Prepared', 'df_cleaned.csv' ))
df_ans= pd.read_csv(os.path.join(path, "Original", 'Dataset-Answers-Weather_Prediction_Pleasant_Weather.csv'))

# Wrangling

In [787]:
# Extract stations list

stations = [col.split('_')[0] for col in df.columns if '_' in col]

# Create a set of unique station names

unique_stations = set(stations)
unique_stations

{'BASEL',
 'BELGRADE',
 'BUDAPEST',
 'DEBILT',
 'DUSSELDORF',
 'HEATHROW',
 'KASSEL',
 'LJUBLJANA',
 'MAASTRICHT',
 'MADRID',
 'MUNCHENB',
 'OSLO',
 'SONNBLICK',
 'STOCKHOLM',
 'VALENTIA'}

In [788]:
# Ensure the DATE column is in string format, if it's not already
#df_cleaned['DATE'] = df_cleaned['DATE'].astype(str)

# Extract the year part from the DATE column
#df_cleaned['Year'] = df_cleaned['DATE'].str[:4].astype(int)

# Filter for the years between 1980 and 1989
#df_filtered = df_cleaned[(df_cleaned['Year'] >= 1980) & (df_cleaned['Year'] <= 1989)]

# Drop the 'Year' column if you don't need it
#df_filtered = df_filtered.drop(columns=['Year'])

In [789]:
# Ensure the DATE column is in string format, if it's not already
#df_ans['DATE'] = df_ans['DATE'].astype(str)

# Extract the year part from the DATE column
#df_ans['Year'] = df_ans['DATE'].str[:4].astype(int)

# Filter for the years between 1980 and 1989
#df_ans_filtered = df_ans[(df_ans['Year'] >= 1980) & (df_ans['Year'] <= 1989)]

# Drop the 'Year' column if you don't need it
#df_ans_filtered = df_ans_filtered.drop(columns=['Year'])

In [790]:
# Drop the specified columns from df_ans_filtered and df_filtered
df_ans = df_ans.drop(columns=['DATE'])
df = df.drop(columns=['Unnamed: 0', 'DATE', 'MONTH'])

In [791]:
df_ans.head(10)

,BASEL_pleasant_weather,BELGRADE_pleasant_weather,BUDAPEST_pleasant_weather,DEBILT_pleasant_weather,DUSSELDORF_pleasant_weather,HEATHROW_pleasant_weather,KASSEL_pleasant_weather,LJUBLJANA_pleasant_weather,MAASTRICHT_pleasant_weather,MADRID_pleasant_weather,MUNCHENB_pleasant_weather,OSLO_pleasant_weather,SONNBLICK_pleasant_weather,STOCKHOLM_pleasant_weather,VALENTIA_pleasant_weather
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [792]:
df

,BASEL_cloud_cover,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_sunshine,BASEL_temp_mean,BASEL_temp_min,BASEL_temp_max,BELGRADE_cloud_cover,...,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max,Year,KASSEL_cloud_cover,MUNCHENB_pressure,STOCKHOLM_humidity
0,7,0.85,1.0180,0.32,0.09,0.7,6.5,0.8,10.9,1,...,0.45,0.34,4.7,8.5,6.0,10.9,1960,8,1.0304,0.98
1,6,0.84,1.0180,0.36,1.05,1.1,6.1,3.3,10.1,6,...,0.25,0.84,0.7,8.9,5.6,12.1,1960,6,1.0292,0.62
2,8,0.90,1.0180,0.18,0.30,0.0,8.5,5.1,9.9,6,...,0.17,0.08,0.1,10.5,8.1,12.9,1960,8,1.0320,0.69
3,3,0.92,1.0180,0.58,0.00,4.1,6.3,3.8,10.6,8,...,0.13,0.98,0.0,7.4,7.3,10.6,1960,6,1.0443,0.98
4,6,0.95,1.0180,0.65,0.14,5.4,3.0,-0.7,6.0,8,...,0.46,0.00,5.7,5.7,3.0,8.4,1960,7,1.0430,0.96
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22945,1,0.79,1.0248,1.34,0.22,7.7,15.9,11.4,21.4,2,...,1.13,0.41,3.4,10.7,7.9,13.5,2022,4,1.0263,0.98
22946,6,0.77,1.0244,1.34,0.22,5.4,16.7,14.3,21.9,0,...,1.13,0.41,3.4,10.7,7.9,13.5,2022,3,1.0263,1.00
22947,4,0.76,1.0227,1.34,0.22,6.1,16.7,13.1,22.4,2,...,1.13,0.41,3.4,10.7,7.9,13.5,2022,3,1.0263,0.85
22948,5,0.80,1.0212,1.34,0.22,5.8,15.4,11.6,21.1,1,...,1.13,0.41,3.4,10.7,7.9,13.5,2022,3,1.0263,0.94


In [793]:
df_ans

,BASEL_pleasant_weather,BELGRADE_pleasant_weather,BUDAPEST_pleasant_weather,DEBILT_pleasant_weather,DUSSELDORF_pleasant_weather,HEATHROW_pleasant_weather,KASSEL_pleasant_weather,LJUBLJANA_pleasant_weather,MAASTRICHT_pleasant_weather,MADRID_pleasant_weather,MUNCHENB_pleasant_weather,OSLO_pleasant_weather,SONNBLICK_pleasant_weather,STOCKHOLM_pleasant_weather,VALENTIA_pleasant_weather
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22945,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
22946,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
22947,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
22948,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [794]:
df_cleaned=df.drop(columns=['Year'])
df_cleaned.shape

(22950, 135)

In [795]:
df_cleaned.shape

(22950, 135)

In [796]:
total_missing = df.isnull().sum().sum()

In [797]:
total_missing

0

In [798]:
df_cleaned.shape

(22950, 135)

# 

In [799]:
# Initialize a dictionary to count observations for each base
base_counts = defaultdict(lambda: defaultdict(int))

# Assuming `columns_list` is already defined and contains the column names
for col in columns_list:
    if '_' in col:  # Ensure column has the format BASE_observation
        base, observation = col.split('_', 1)
        base_counts[base][observation] += 1

# Prepare the data to calculate the total count for each base
total_counts = {}

# Calculate total counts for each base
for base, observations in base_counts.items():
    total_counts[base] = sum(observations.values())  # Total number of observations for each base

# Convert the total counts to a DataFrame for easy readability
df_total_counts = pd.DataFrame(list(total_counts.items()), columns=['Base', 'Total Observation Count'])

# Display the DataFrame
print(df_total_counts)

          Base  Total Observation Count
0        BASEL                        9
1     BELGRADE                        9
2     BUDAPEST                        9
3       DEBILT                        9
4   DUSSELDORF                        9
5     HEATHROW                        9
6       KASSEL                        9
7    LJUBLJANA                        9
8   MAASTRICHT                        9
9       MADRID                        9
10    MUNCHENB                        9
11        OSLO                        9
12   SONNBLICK                        9
13   STOCKHOLM                        9
14    VALENTIA                        9


## Add missing oberservation

In [800]:
# Create a list of all unique station names in the dataset

all_stations = set([col.split('_')[0] for col in df_cleaned.columns if '_' in col])
all_stations

{'BASEL',
 'BELGRADE',
 'BUDAPEST',
 'DEBILT',
 'DUSSELDORF',
 'HEATHROW',
 'KASSEL',
 'LJUBLJANA',
 'MAASTRICHT',
 'MADRID',
 'MUNCHENB',
 'OSLO',
 'SONNBLICK',
 'STOCKHOLM',
 'VALENTIA'}

In [801]:
# Create a list of all unique station names in the dataset

all_stations = set([col.split('_')[0] for col in df_cleaned.columns if '_' in col])
all_stations

{'BASEL',
 'BELGRADE',
 'BUDAPEST',
 'DEBILT',
 'DUSSELDORF',
 'HEATHROW',
 'KASSEL',
 'LJUBLJANA',
 'MAASTRICHT',
 'MADRID',
 'MUNCHENB',
 'OSLO',
 'SONNBLICK',
 'STOCKHOLM',
 'VALENTIA'}

In [802]:
#df_cleaned=df.drop(columns=['DATE', 'MONTH'])
#df_ans=df_ans.drop(columns=['DATE'])

# Reshape Data

In [803]:
df_cleaned.shape

(22950, 135)

In [804]:
df_cleaned.head(10)

,BASEL_cloud_cover,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_sunshine,BASEL_temp_mean,BASEL_temp_min,BASEL_temp_max,BELGRADE_cloud_cover,...,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max,KASSEL_cloud_cover,MUNCHENB_pressure,STOCKHOLM_humidity
0,7,0.85,1.018,0.32,0.09,0.7,6.5,0.8,10.9,1,...,1.0003,0.45,0.34,4.7,8.5,6.0,10.9,8,1.0304,0.98
1,6,0.84,1.018,0.36,1.05,1.1,6.1,3.3,10.1,6,...,1.0007,0.25,0.84,0.7,8.9,5.6,12.1,6,1.0292,0.62
2,8,0.90,1.018,0.18,0.30,0.0,8.5,5.1,9.9,6,...,1.0096,0.17,0.08,0.1,10.5,8.1,12.9,8,1.0320,0.69
3,3,0.92,1.018,0.58,0.00,4.1,6.3,3.8,10.6,8,...,1.0184,0.13,0.98,0.0,7.4,7.3,10.6,6,1.0443,0.98
4,6,0.95,1.018,0.65,0.14,5.4,3.0,-0.7,6.0,8,...,1.0328,0.46,0.00,5.7,5.7,3.0,8.4,7,1.0430,0.96
5,1,0.78,1.018,0.76,0.00,7.7,1.5,-1.8,6.4,8,...,1.0326,0.33,0.00,0.9,4.1,-0.6,8.7,7,1.0338,0.72
6,3,0.87,1.018,0.66,0.00,5.4,-0.4,-4.5,6.1,8,...,1.0230,0.42,0.00,2.2,7.9,4.4,11.4,4,1.0373,0.87
7,8,0.95,1.018,0.19,0.09,0.0,0.9,-2.2,2.6,6,...,1.0203,0.50,0.00,6.2,8.5,6.7,10.3,8,1.0315,0.50
8,8,0.98,1.018,0.19,0.56,0.0,-2.4,-4.9,1.5,8,...,1.0243,0.14,0.00,0.1,6.8,5.8,7.8,8,1.0301,0.53
9,3,0.74,1.018,0.38,0.01,1.1,-6.9,-10.6,-3.4,8,...,1.0307,0.49,0.00,6.4,1.8,0.2,5.0,3,1.0301,0.56


In [805]:
#Create an 'X' matrix by dropping the unneeded columns.
X = df_cleaned
y = df_ans

In [806]:
X.shape

(22950, 135)

In [807]:
# Turn X and y from a df to arrays

X = np.array(X)
y = np.array(y)

In [808]:
# reshape
X = X.reshape(-1,15,9)

In [809]:
# Check Shape

X.shape

(22950, 15, 9)

In [810]:
# Use argmax to transform y
y =  np.argmax(y, axis = 1)
y

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [811]:
y.shape

(22950,)

In [812]:
# Check y layout

from sklearn.utils.multiclass import type_of_target
type_of_target(y)

'multiclass'

# Data Split

In [813]:
# Split data into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X,y,random_state = 42)

In [814]:
print(X_train.shape, "|", Y_train.shape)
print(X_test.shape, "|", Y_test.shape)

(17212, 15, 9) | (17212,)
(5738, 15, 9) | (5738,)


In [815]:
#tf.random.set_seed(42)

In [816]:
#X_train

In [817]:
#len(X_train[0])

In [818]:
#len(X_train[0][0])

# Bayesian Hyperparameter Optimized

In [819]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = 15 # Number of weather stations
# Make scorer accuracy
score_acc = make_scorer(accuracy_score)

In [820]:
# Create function

def bay_area(neurons, activation, kernel, optimizer, learning_rate, batch_size, epochs,
              layers1, layers2, normalization, dropout, dropout_rate): 
    optimizerL = ['SGD', 'Adam', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl','SGD']
    #optimizerD= {'Adam':Adam(lr=learning_rate), 'SGD':SGD(lr=learning_rate),
                 #'RMSprop':RMSprop(lr=learning_rate), 'Adadelta':Adadelta(lr=learning_rate),
                 #'Adagrad':Adagrad(lr=learning_rate), 'Adamax':Adamax(lr=learning_rate),
                 #'Nadam':Nadam(lr=learning_rate), 'Ftrl':Ftrl(lr=learning_rate)}
    activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu',
                   'elu', 'exponential', LeakyReLU,'relu']
    
    neurons = round(neurons)
    kernel = round(kernel)
    activation = activationL[round(activation)]  #optimizerD[optimizerL[round(optimizer)]]
    optimizer = optimizerL[round(optimizer)]
    batch_size = round(batch_size)
    
    epochs = round(epochs)
    layers1 = round(layers1)
    layers2 = round(layers2)
    
    def cnn_model():
        model = Sequential()
        model.add(Conv1D(neurons, kernel_size=kernel,activation=activation, input_shape=(timesteps, input_dim)))
        #model.add(Conv1D(32, kernel_size=1,activation='relu', input_shape=(timesteps, input_dim)))
        
        if normalization > 0.5:
            model.add(BatchNormalization())
        for i in range(layers1):
            model.add(Dense(neurons, activation=activation)) #(neurons, activation=activation))
        if dropout > 0.5:
            model.add(Dropout(dropout_rate, seed=123))
        for i in range(layers2):
            model.add(Dense(neurons, activation=activation))
        model.add(MaxPooling1D())
        model.add(Flatten())
        model.add(Dense(n_classes, activation='softmax')) #sigmoid softmax
        #model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy']) #categorical_crossentropy
        model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy']) #categorical_crossentropy
        return model
    es = EarlyStopping(monitor='accuracy', mode='max', verbose=2, patience=20)
    nn = KerasClassifier(build_fn=cnn_model, epochs=epochs, batch_size=batch_size, verbose=2)
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)
    score = cross_val_score(nn, X_train, Y_train, scoring=score_acc, cv=kfold, params={'callbacks':[es]}).mean()
    return score

In [821]:
start = time.time()
params ={
    'neurons': (10, 100),
    'kernel': (1, 3),
    'activation':(0, 9), 
    'optimizer':(0,7),
    'learning_rate':(0.01, 1),
    'batch_size': (200, 1000), 
    'epochs':(20, 50),
    'layers1':(1,3),
    'layers2':(1,3),
    'normalization':(0,1),
    'dropout':(0,1),
    'dropout_rate':(0,0.3)
}
# Run Bayesian Optimization
nn_opt = BayesianOptimization(bay_area, params, random_state=42)
nn_opt.maximize(init_points=15, n_iter=4) 
print('Search took %s minutes' % ((time.time() - start)/60))

|   iter    |  target   | activa... | batch_... |  dropout  | dropou... |  epochs   |  kernel   |  layers1  |  layers2  | learni... |  neurons  | normal... | optimizer |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------


c:\Users\Oa\anaconda3\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\Oa\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/25


15/15 - 14s - 957ms/step - accuracy: 0.5998 - loss: 2.7067
Epoch 2/25
15/15 - 2s - 166ms/step - accuracy: 0.6440 - loss: 2.7004
Epoch 3/25
15/15 - 3s - 173ms/step - accuracy: 0.6440 - loss: 2.6971
Epoch 4/25
15/15 - 3s - 167ms/step - accuracy: 0.6440 - loss: 2.6942
Epoch 5/25


KeyboardInterrupt: 

In [ ]:
####Need to add learning rate to the optimizer type??
epochs = 38
batch_size = 828
#n_hidden = 32

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = 15 #_count_classes(Y_train)
layers1 = 2
layers2 = 1
activation = 'softsign'
kernel =  int(round(1.093))  # Rounded kernel size for Conv1D
neurons = 95
normalization = 0.9656
dropout = 0.7296
dropout_rate = 0.1912
optimizer = Adadelta(learning_rate=0.0744)  # Instantiate RMSprop with learning rate

model = Sequential()
model.add(Conv1D(neurons, kernel_size=kernel, activation=activation, input_shape=(timesteps, input_dim)))
if normalization > 0.5:
    model.add(BatchNormalization())
for i in range(layers1):
    model.add(Dense(neurons, activation=activation))
if dropout > 0.5:
    model.add(Dropout(dropout_rate, seed=123))
for i in range(layers2):
    model.add(Dense(neurons, activation=activation))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='softmax')) #softmax sigmoid
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy']) #binary_crossentropy

In [824]:
optimum = nn_opt.max['params']
learning_rate = optimum['learning_rate']

activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu', 'elu', 'exponential', LeakyReLU, 'relu']
optimum['activation'] = activationL[round(optimum['activation'])]

optimum['batch_size'] = round(optimum['batch_size'])
optimum['epochs'] = round(optimum['epochs'])
optimum['layers1'] = round(optimum['layers1'])
optimum['layers2'] = round(optimum['layers2'])
optimum['neurons'] = round(optimum['neurons'])

optimizerL = ['Adam', 'SGD', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl', 'Adam']
optimizerD = {
    'Adam': Adam(learning_rate=learning_rate),
    'SGD': SGD(learning_rate=learning_rate),
    'RMSprop': RMSprop(learning_rate=learning_rate),
    'Adadelta': Adadelta(learning_rate=learning_rate),
    'Adagrad': Adagrad(learning_rate=learning_rate),
    'Adamax': Adamax(learning_rate=learning_rate),
    'Nadam': Nadam(learning_rate=learning_rate),
    'Ftrl': Ftrl(learning_rate=learning_rate)
}
optimum['optimizer'] = optimizerD[optimizerL[round(optimum['optimizer'])]]
optimum

TypeError: 'NoneType' object is not subscriptable

In [828]:
model.summary()

Model: "sequential_104"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_104 (Conv1D)             │ (None, 15, 95)         │           950 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_43          │ (None, 15, 95)         │           380 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_499 (Dense)               │ (None, 15, 95)         │         9,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_500 (Dense)               │ (None, 15, 95)         │         9,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_58 (Dropout)            │ (None, 15, 95)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_501 (Dense)               │ (None, 15, 95)         │         9,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_101               │ (None, 7, 95)          │             0 │
│ (MaxPooling1D)                  │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_101 (Flatten)           │ (None, 665)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_502 (Dense)               │ (None, 15)             │         9,990 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 38,680 (151.09 KB)

 Trainable params: 38,490 (150.35 KB)

 Non-trainable params: 190 (760.00 B)

In [830]:
# Extract all unique observations from the column names
unique_observations = set([col.split('_', 1)[1] for col in df.columns if '_' in col])

# Display the unique observations
print(unique_observations)

{'global_radiation', 'cloud_cover', 'temp_mean', 'temp_min', 'temp_max', 'precipitation', 'humidity', 'sunshine', 'pressure'}


In [838]:
# Put the y_test set back into a one-hot configuration

y_train_one_hot = to_categorical(Y_train, num_classes=15)

In [839]:
# Check shapes

print(f'X_train shape: {X_train.shape}')
print(f'y_train_one_hot shape: {y_train_one_hot.shape}')

X_train shape: (17212, 15, 9)
y_train_one_hot shape: (17212, 15)


In [841]:
# Compile the model with categorical_crossentropy

model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [842]:
# Fit the model to the data

model.fit(X_train, y_train_one_hot, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/38
21/21 - 49s - 2s/step - accuracy: 0.7366 - loss: 0.7809
Epoch 2/38
21/21 - 5s - 250ms/step - accuracy: 0.7375 - loss: 0.7775
Epoch 3/38
21/21 - 5s - 251ms/step - accuracy: 0.7390 - loss: 0.7728
Epoch 4/38
21/21 - 3s - 149ms/step - accuracy: 0.7421 - loss: 0.7685
Epoch 5/38
21/21 - 4s - 174ms/step - accuracy: 0.7423 - loss: 0.7654
Epoch 6/38
21/21 - 3s - 158ms/step - accuracy: 0.7439 - loss: 0.7597
Epoch 7/38
21/21 - 3s - 162ms/step - accuracy: 0.7438 - loss: 0.7555
Epoch 8/38
21/21 - 4s - 213ms/step - accuracy: 0.7475 - loss: 0.7508
Epoch 9/38
21/21 - 4s - 185ms/step - accuracy: 0.7481 - loss: 0.7483
Epoch 10/38
21/21 - 3s - 153ms/step - accuracy: 0.7485 - loss: 0.7447
Epoch 11/38
21/21 - 3s - 151ms/step - accuracy: 0.7507 - loss: 0.7401
Epoch 12/38
21/21 - 4s - 186ms/step - accuracy: 0.7519 - loss: 0.7363
Epoch 13/38
21/21 - 6s - 299ms/step - accuracy: 0.7513 - loss: 0.7352
Epoch 14/38
21/21 - 4s - 182ms/step - accuracy: 0.7519 - loss: 0.7314
Epoch 15/38
21/21 - 3s - 165ms/

# Compiling and Running

In [861]:
# Define list of stations names

stations = {
0: 'BASEL',
1: 'BELGRADE',
2: 'BUDAPEST',
3: 'DEBILT',
4: 'DUSSELDORF',
5: 'HEATHROW',
6: 'KASSEL',
7: 'LJUBLJANA',
8: 'MAASTRICHT',
9: 'MADRID',
10: 'MUNCHENB',
11: 'OSLO',
12: 'SONNBLICK',
13: 'STOCKHOLM',
14: 'VALENTIA'

}

In [862]:
def confusion_matrix(Y_true, Y_pred, stations):
    Y_true = pd.Series([activities[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([activities[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

In [865]:
def confusion_matrix(y_true, y_pred, stations):
    # Check if y_true and y_pred are one-hot encoded or already class indices
    if y_true.ndim == 1:
        y_true_labels = y_true
    else:
        y_true_labels = np.argmax(y_true, axis=1)
    
    if y_pred.ndim == 1:
        y_pred_labels = y_pred
    else:
        y_pred_labels = np.argmax(y_pred, axis=1)
        
    # Map numeric labels to activity names
    y_true_series = pd.Series([stations[y] for y in y_true_labels])
    y_pred_series = pd.Series([stations[y] for y in y_pred_labels])
    
    return pd.crosstab(y_true_series, y_pred_series, rownames=['True'], colnames=['Pred'])

In [855]:
from tensorflow.keras.utils import to_categorical  # Import the function

ty = to_categorical(Y_test)  # Use the correct function
print(ty)

[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]]


In [866]:
Y_pred = model.predict(X_test)

180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step


In [867]:
# Evaluate

print(confusion_matrix(Y_test, Y_pred, stations))

Pred        BASEL  BELGRADE  BUDAPEST  DEBILT  HEATHROW  MADRID
True                                                           
BASEL        3397       210         9       3         2      61
BELGRADE      323       739         8       0         0      22
BUDAPEST       73        81        36       2         1      21
DEBILT         37        21         8      11         2       3
DUSSELDORF     15         6         4       0         1       3
HEATHROW       32         9         4       2         4      31
KASSEL          6         3         1       0         0       1
LJUBLJANA      20        13         3       0         0      25
MAASTRICHT      9         0         0       0         0       0
MADRID        147        53         7       0         2     249
MUNCHENB        7         1         0       0         0       0
OSLO            1         1         0       0         0       3
STOCKHOLM       1         2         1       0         0       0
VALENTIA        1         0         0   